In [1]:
## Instalaciones

%pip install torch
%pip install open3d

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
## Dependencias

import torch
import os
import open3d as o3d
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader
from model import PointnetClassifier, PointNetLoss
from modelnet10 import ModelNetClass, ModelNet, DatasetType
from utils.csv import save_loss_dict

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {DEVICE}.")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Using cuda.


In [ ]:
# parametros de ejecución
checkpoint_freq = 10        # cada cuantos epoch guardar el modelo
ROOT_DIR = os.getcwd()
CHECKPOINT_DIR = os.path.join(ROOT_DIR, "checkpoint")

# parametros del dataset
classes = [ModelNetClass.MONITOR, ModelNetClass.TABLE, ModelNetClass.TOILET]
batch_size = 32
dim = 3
num_points = 1024
num_classes = len(classes)

# hiperparametros
num_global_feats = 1024     # número de features globales calculadas
epochs = 500
learning_rate = 0.01
reg_weight = 0.001


train_data = ModelNet(classes, DatasetType.TRAIN)
validation_data = ModelNet(classes, DatasetType.VALIDATION)
test_data = ModelNet(classes, DatasetType.TEST)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_data, batch_size=batch_size)

classifier = PointnetClassifier(dim, num_points, num_global_feats, num_classes).to(DEVICE)

optimizer = optim.Adam(classifier.parameters(), lr=learning_rate)
criterion = PointNetLoss(alpha=None, gamma=1, reg_weight=reg_weight, size_average=True).to(DEVICE)

loss_dict = {
    "train": {
        "loss": list(),
        "acc": list()
    },
    "valid": {
        "loss": list(),
        "acc": list()
    }
}

for epoch in range(1, epochs + 1):
    print(f"Epoch {epoch}:")
    '''
    Entrenamiento
    '''
    classifier = classifier.train()

    train_batch_losses = list()
    train_batch_accs = list()
    
    for pcds, labels in train_loader:
        pcds = pcds.to(DEVICE)
        labels = labels.squeeze().to(DEVICE)

        # Gradientes en cero
        optimizer.zero_grad()
        
        # Hacer predicciones
        out, _, A = classifier(pcds)
        loss = criterion(out, labels, A)
                
        # Calcular gradiente y optimizar
        loss.backward()
        optimizer.step()

        # Calculamos las elecciones
        pred_choice = torch.softmax(out, dim=1).argmax(dim=1)
        
        # Elecciones correctas
        correct = pred_choice.eq(labels.data).cpu().sum()
        accuracy = correct.item() / float(pcds.size(0))

        # Registramos métricas para el batch
        train_batch_losses.append(loss.item() * pcds.size(0))
        train_batch_accs.append(accuracy)
        
    epoch_train_loss = np.mean(train_batch_losses)
    epoch_train_acc = np.mean(train_batch_accs)
    print("  Train Loss:\t\t", epoch_train_loss)
    print("  Train Acc:\t\t", epoch_train_acc)
    loss_dict["train"]["loss"].append(epoch_train_loss)
    loss_dict["train"]["acc"].append(epoch_train_acc)

    '''
    Validación
    '''
    with torch.no_grad():
        classifier = classifier.eval()

        valid_batch_losses = list()
        valid_batch_accs = list()
        
        for pcds, labels in validation_loader:
            pcds = pcds.to(DEVICE)
            labels = labels.squeeze().to(DEVICE)

            # Gradientes en cero
            optimizer.zero_grad()
            
            # Hacer predicciones
            out, _, A = classifier(pcds)
            loss = criterion(out, labels, A)

            # Calculamos las elecciones
            pred_choice = torch.softmax(out, dim=1).argmax(dim=1)
            
            # Elecciones correctas
            correct = pred_choice.eq(labels.data).cpu().sum()
            accuracy = correct.item() / float(pcds.size(0))

            # Registramos métricas para el batch
            valid_batch_losses.append(loss.item() * pcds.size(0))
            valid_batch_accs.append(accuracy)
            
    epoch_valid_loss = np.mean(valid_batch_losses)
    epoch_valid_acc = np.mean(valid_batch_accs)
    print("  Validation Loss:\t", epoch_valid_loss)
    print("  Validation Acc:\t", epoch_valid_acc)
    loss_dict["valid"]["loss"].append(epoch_valid_loss)
    loss_dict["valid"]["acc"].append(epoch_valid_acc)

    if epoch % checkpoint_freq == 0:
        path = os.path.join(CHECKPOINT_DIR, f"model_epoch_{str(epoch).zfill(4)}.pth")
        torch.save(classifier.state_dict(), path)

save_loss_dict(loss_dict)

Epoch 1:
  Train Loss:		 52.281550005862584
  Train Acc:		 0.4581075851393188
  Validation Loss:	 9249.91427230835
  Validation Acc:	 0.36328125
Epoch 2:
  Train Loss:		 38.776853454740426
  Train Acc:		 0.6001838235294118
  Validation Loss:	 625021.2314453125
  Validation Acc:	 0.3046875
Epoch 3:
  Train Loss:		 25.424504349106236
  Train Acc:		 0.6509771671826625
  Validation Loss:	 469073.3698730469
  Validation Acc:	 0.359375
Epoch 4:
  Train Loss:		 27.556997976805036
  Train Acc:		 0.6815015479876161
  Validation Loss:	 1330243.6440429688
  Validation Acc:	 0.328125
Epoch 5:
  Train Loss:		 23.761399987496826
  Train Acc:		 0.7052534829721363
  Validation Loss:	 2700818.271484375
  Validation Acc:	 0.3671875
Epoch 6:
  Train Loss:		 22.47807680776245
  Train Acc:		 0.7528541021671827
  Validation Loss:	 323717.9020996094
  Validation Acc:	 0.30755208333333334
Epoch 7:
  Train Loss:		 16.563395156672126
  Train Acc:		 0.7762674148606812
  Validation Loss:	 374251.6022949219
  Vali